In [1]:
#q1

!hdfs dfsadmin -fs hdfs://boss:9000 -report | grep "Live datanodes"

Live datanodes (2):


In [2]:
!wget -nc https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv


!hdfs dfs -rm -f hdfs://boss:9000/single.csv
!hdfs dfs -rm -f hdfs://boss:9000/double.csv

!hdfs dfs -D dfs.blocksize=1048576 -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://boss:9000/single.csv

!hdfs dfs -D dfs.blocksize=1048576 -D dfs.replication=2 -cp hdma-wi-2021.csv hdfs://boss:9000/double.csv


File ‘hdma-wi-2021.csv’ already there; not retrieving.



In [3]:
#q2
!hdfs dfs -du -h hdfs://boss:9000/single.csv hdfs://boss:9000/double.csv


166.8 M  166.8 M  hdfs://boss:9000/single.csv
166.8 M  333.7 M  hdfs://boss:9000/double.csv


In [4]:
#q3
import requests

url = f"http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS"
response = requests.get(url)

response.json()


{'FileStatus': {'accessTime': 1710824327694,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16386,
  'group': 'supergroup',
  'length': 174944099,
  'modificationTime': 1710824333012,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [5]:
#q4

url = f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true"
response = requests.get(url)
response.json()['Location']


'http://4b4a0ab92fe9:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0'

In [6]:
#q5
def get_block_distribution(filename):
    url = f"http://boss:9870/webhdfs/v1/{filename}?op=GET_BLOCK_LOCATIONS"
    response = requests.get(url)
    blocks = response.json()['LocatedBlocks']['locatedBlocks']
    distribution = {}
    for block in blocks:
        for location in block['locations']:
            container_id = location['hostName'].split(":")[0]
            distribution[container_id] = distribution.get(container_id, 0) + 1
    return distribution

block_distribution = get_block_distribution("single.csv")
block_distribution

{'4b4a0ab92fe9': 81, '53fc81fce6c8': 86}

In [7]:
#q6
import pyarrow as pa
import pyarrow.fs

hdfs = pa.fs.HadoopFileSystem("boss", 9000)
with hdfs.open_input_file("/single.csv") as f:
    result = f.read_at(10, 0)


result


2024-03-19 04:59:10,205 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


b'activity_y'

In [8]:
#q7

hdfs = pa.fs.HadoopFileSystem(host="boss", port=9000)
count = 0
with hdfs.open_input_file("/single.csv") as f:
    content = f.read().decode('utf-8') 
    for line in content.splitlines():
        if "Single Family" in line:
            count += 1
count

444874